In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType, StructType, StringType, IntegerType, FloatType, StructField
from pyspark.sql.functions import lit, concat, col, regexp_replace, collect_list

In [3]:
ss = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("us_import_sample") \
    .getOrCreate()

In [4]:
cargo_desc = ss.read \
    .option("header", True) \
    .option("escape", '"') \
    .option("inferSchema", True) \
    .csv('./ams/2020/202001201500/ams__cargodesc_2020__202001201500.csv')

In [5]:
cargo_desc.printSchema()

root
 |-- identifier: long (nullable = true)
 |-- container_number: string (nullable = true)
 |-- description_sequence_number: integer (nullable = true)
 |-- piece_count: integer (nullable = true)
 |-- description_text: string (nullable = true)



In [6]:
cargo_desc.createOrReplaceTempView("cargo_desc")

In [7]:
hazmat = ss.read \
    .option("header", True) \
    .option("escape", '"') \
    .option("inferSchema", True) \
    .csv('./ams/2020/202001201500/ams__hazmat_2020__202001201500.csv')

In [8]:
hazmat.printSchema()

root
 |-- identifier: long (nullable = true)
 |-- container_number: string (nullable = true)
 |-- hazmat_sequence_number: integer (nullable = true)
 |-- hazmat_code: string (nullable = true)
 |-- hazmat_class: string (nullable = true)
 |-- hazmat_code_qualifier: string (nullable = true)
 |-- hazmat_contact: string (nullable = true)
 |-- hazmat_page_number: string (nullable = true)
 |-- hazmat_flash_point_temperature: string (nullable = true)
 |-- hazmat_flash_point_temperature_negative_ind: string (nullable = true)
 |-- hazmat_flash_point_temperature_unit: string (nullable = true)
 |-- hazmat_description: string (nullable = true)



In [9]:
hazmat.createOrReplaceTempView("hazmat")

In [10]:
hazmat_class = ss.read \
    .option("header", True) \
    .option("escape", '"') \
    .option("inferSchema", True) \
    .csv('./ams/2020/202001201500/ams__hazmatclass_2020__202001201500.csv')

In [11]:
hazmat_class.printSchema()

root
 |-- identifier: long (nullable = true)
 |-- container_number: string (nullable = true)
 |-- hazmat_sequence_number: integer (nullable = true)
 |-- hazmat_classification: string (nullable = true)



In [12]:
hazmat_class.createOrReplaceTempView("hazmat_class")

In [14]:
tariff = ss.read \
    .option("header", True) \
    .option("escape", '"') \
    .option("inferSchema", True) \
    .csv('./ams/2020/202001201500/ams__tariff_2020__202001201500.csv')

In [15]:
tariff.printSchema()

root
 |-- identifier: long (nullable = true)
 |-- container_number: string (nullable = true)
 |-- description_sequence_number: integer (nullable = true)
 |-- harmonized_number: long (nullable = true)
 |-- harmonized_value: double (nullable = true)
 |-- harmonized_weight: integer (nullable = true)
 |-- harmonized_weight_unit: string (nullable = true)



In [16]:
tariff.createOrReplaceTempView("tariff")

In [25]:
schedule = ss.read \
    .option("header", True) \
    .option("escape", '"') \
    .option("inferSchema", True) \
    .csv('./ams/hts.csv')

In [26]:
schedule.printSchema()

root
 |-- hts_number: string (nullable = true)
 |-- indent: string (nullable = true)
 |-- description: string (nullable = true)
 |-- unit_of_quantity: string (nullable = true)
 |-- general_rate_of_duty: string (nullable = true)
 |-- special_rate_of_duty: string (nullable = true)
 |-- column_2_rate_of_duty: string (nullable = true)
 |-- quota_quantity: string (nullable = true)
 |-- additional_duties: string (nullable = true)



In [27]:
schedule.createOrReplaceTempView("tariff_schedule")

In [32]:
cargo_table = ss.sql("""
    SELECT 
        c.identifier,
        c.container_number,
        c.description_sequence_number AS sequence_number,
        c.piece_count,
        c.description_text AS description,
        h.hazmat_code,
        (CASE 
            WHEN (hc.hazmat_classification IS NOT NULL) THEN hc.hazmat_classification
            ELSE h.hazmat_class
        END) AS hazmat_class,
        h.hazmat_code_qualifier,
        h.hazmat_contact,
        h.hazmat_page_number,
        h.hazmat_flash_point_temperature,
        h.hazmat_flash_point_temperature_negative_ind,
        h.hazmat_flash_point_temperature_unit,
        h.hazmat_description,
        t.harmonized_number,
        t.harmonized_value,
        t.harmonized_weight,
        t.harmonized_weight_unit,
        ts.description AS harmonized_tariff_schedule_desc,
        ts.general_rate_of_duty,
        ts.special_rate_of_duty,
        ts.column_2_rate_of_duty,
        ts.quota_quantity,
        ts.additional_duties
    FROM cargo_desc AS c
    LEFT JOIN hazmat AS h
    ON 
        c.identifier = h.identifier AND 
        c.container_number = h.container_number AND 
        c.description_sequence_number = h.hazmat_sequence_number
    LEFT JOIN hazmat_class AS hc
    ON
        c.identifier = hc.identifier AND 
        c.container_number = hc.container_number AND 
        c.description_sequence_number = hc.hazmat_sequence_number
    LEFT JOIN tariff as t
    ON
        c.identifier = t.identifier AND 
        c.container_number = t.container_number AND 
        c.description_sequence_number = t.description_sequence_number
    LEFT JOIN tariff_schedule as ts
    ON
        CAST(t.harmonized_number as string) = ts.hts_number
""")

In [33]:
cargo_table.printSchema()

root
 |-- identifier: long (nullable = true)
 |-- container_number: string (nullable = true)
 |-- sequence_number: integer (nullable = true)
 |-- piece_count: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- hazmat_code: string (nullable = true)
 |-- hazmat_class: string (nullable = true)
 |-- hazmat_code_qualifier: string (nullable = true)
 |-- hazmat_contact: string (nullable = true)
 |-- hazmat_page_number: string (nullable = true)
 |-- hazmat_flash_point_temperature: string (nullable = true)
 |-- hazmat_flash_point_temperature_negative_ind: string (nullable = true)
 |-- hazmat_flash_point_temperature_unit: string (nullable = true)
 |-- hazmat_description: string (nullable = true)
 |-- harmonized_number: long (nullable = true)
 |-- harmonized_value: double (nullable = true)
 |-- harmonized_weight: integer (nullable = true)
 |-- harmonized_weight_unit: string (nullable = true)
 |-- harmonized_tariff_schedule_desc: string (nullable = true)
 |-- general_rate_of